In [48]:
import numpy as np
import copy
import pickle


def LoadBatch(filename):
    """ Copied from the dataset website """
    with open('Datasets/' + filename, 'rb') as fo:
        dataset_dict = pickle.load(fo, encoding='bytes')
 
    return dataset_dict

def compute_grads_num_slow__(X, Y, net_params, lambda_, h):
    L = len(net_params) // 2
    grads = {}
    for l in range(1, L + 1):
        grads['W' + str(l)] = None
        grads['b' + str(l)] = None

    if net_params.get('use_bn'):
        for l in range(1, L):
            grads['gamma' + str(l)] = None
            grads['beta' + str(l)] = None

        
    if net_params['use_bn']:
        grads['gammas'] = [None] * len(net_params['gammas'])
        grads['betas'] = [None] * len(net_params['betas'])
    
    for j in range(len(net_params['b'])):
        grads['b'][j] = np.zeros_like(net_params['b'][j])
        net_try = copy.deepcopy(net_params)
        for i in range(len(net_params['b'][j])):
            b_try = copy.deepcopy(net_params['b'])
            b_try[j][i] -= h
            net_try['b'] = b_try
            c1 = compute_cost(X, Y, net_try, lambda_)
            
            b_try = copy.deepcopy(net_params['b'])
            b_try[j][i] += h
            net_try['b'] = b_try
            c2 = compute_cost(X, Y, net_try, lambda_)
            
            grads['b'][j][i] = (c2 - c1) / (2 * h)

    for j in range(len(net_params['W'])):
        grads['W'][j] = np.zeros_like(net_params['W'][j])
        net_try = copy.deepcopy(net_params)
        for i in np.ndindex(net_params['W'][j].shape):
            W_try = copy.deepcopy(net_params['W'])
            W_try[j][i] -= h
            net_try['W'] = W_try
            c1 = compute_cost(X, Y, net_try, lambda_)
            
            W_try = copy.deepcopy(net_params['W'])
            W_try[j][i] += h
            net_try['W'] = W_try
            c2 = compute_cost(X, Y, net_try, lambda_)
            
            grads['W'][j][i] = (c2 - c1) / (2 * h)

    if net_params['use_bn']:
        for j in range(len(net_params['gammas'])):
            grads['gammas'][j] = np.zeros_like(net_params['gammas'][j])
            net_try = copy.deepcopy(net_params)
            for i in np.ndindex(net_params['gammas'][j].shape):
                gammas_try = copy.deepcopy(net_params['gammas'])
                gammas_try[j][i] -= h
                net_try['gammas'] = gammas_try
                c1 = compute_cost(X, Y, net_try, lambda_)

                gammas_try = copy.deepcopy(net_params['gammas'])
                gammas_try[j][i] += h
                net_try['gammas'] = gammas_try
                c2 = compute_cost(X, Y, net_try, lambda_)
                
                grads['gammas'][j][i] = (c2 - c1) / (2 * h)

        for j in range(len(net_params['betas'])):
            grads['betas'][j] = np.zeros_like(net_params['betas'][j])
            net_try = copy.deepcopy(net_params)
            for i in np.ndindex(net_params['betas'][j].shape):
                betas_try = copy.deepcopy(net_params['betas'])  
                betas_try[j][i] -= h
                net_try['betas'] = betas_try
                c1 = compute_cost(X, Y, net_try, lambda_)

                betas_try = copy.deepcopy(net_params['betas'])
                betas_try[j][i] += h
                net_try['betas'] = betas_try
                c2 = compute_cost(X, Y, net_try, lambda_)

                grads['betas'][j][i] = (c2 - c1) / (2 * h)

        
    return grads


def ComputeGradsNum(X, Y, P, W, b, lamda, h):
	""" Converted from matlab code """
	no 	= 	W.shape[0]
	d 	= 	X.shape[0]

	# grad_W = np.zeros(W.shape);
	# grad_b = np.zeros((no, 1));

	grad_W = np.empty(shape=W.shape, dtype=object)
	grad_b = np.empty(shape=b.shape, dtype=object)

	c = ComputeCost(X, Y, W, b, lamda)
	
	for j in range(len(b)):
		grad_b[j] = np.zeros(shape=(b[j].shape))

		for i in range(len(b[j])):
			b_try = np.array(b)
			b_try[j][i] += h
			c2 = ComputeCost(X, Y, W, b_try, lamda)
			grad_b[j][i] = (c2-c) / h
	
	for k in range(len(W)):
		grad_W[k] = np.zeros(shape=(W[k].shape))

		for i in range(W[k].shape[0]):
			for j in range(W[k].shape[1]):
				W_try = np.array(W)
				W_try[k][i,j] += h
				c2 = ComputeCost(X, Y, W_try, b, lamda)

				grad_W[k][i,j] = (c2-c) / h

	return [grad_W, grad_b]

def compute_grads_num_slow(X, Y, net_params, lambda_, h):
    L = len(net_params) // 2
    grads = {}
    for l in range(1, L + 1):
        grads['W' + str(l)] = np.zeros_like(net_params['W' + str(l)])
        grads['b' + str(l)] = np.zeros_like(net_params['b' + str(l)])

    if net_params.get('use_bn'):
        for l in range(1, L):
            grads['gamma' + str(l)] = np.zeros_like(net_params['gamma' + str(l)])
            grads['beta' + str(l)] = np.zeros_like(net_params['beta' + str(l)])

    for l in range(1, L + 1):
        for i in np.ndindex(net_params['W' + str(l)].shape):
            net_try = copy.deepcopy(net_params)
            net_try['W' + str(l)][i] -= h
            c1 = compute_cost(X, Y, net_try, lambda_)
            
            net_try = copy.deepcopy(net_params)
            net_try['W' + str(l)][i] += h
            c2 = compute_cost(X, Y, net_try, lambda_)
            
            grads['W' + str(l)][i] = (c2 - c1) / (2 * h)

        for i in range(net_params['b' + str(l)].size):
            net_try = copy.deepcopy(net_params)
            net_try['b' + str(l)][i] -= h
            c1 = compute_cost(X, Y, net_try, lambda_)
            
            net_try = copy.deepcopy(net_params)
            net_try['b' + str(l)][i] += h
            c2 = compute_cost(X, Y, net_try, lambda_)
            
            grads['b' + str(l)][i] = (c2 - c1) / (2 * h)

    if net_params['use_bn']:
        for l in range(1, L):
            for i in np.ndindex(net_params['gamma' + str(l)].shape):
                net_try = copy.deepcopy(net_params)
                net_try['gamma' + str(l)][i] -= h
                c1 = compute_cost(X, Y, net_try, lambda_)

                net_try = copy.deepcopy(net_params)
                net_try['gamma' + str(l)][i] += h
                c2 = compute_cost(X, Y, net_try, lambda_)
                
                grads['gamma' + str(l)][i] = (c2 - c1) / (2 * h)

            for i in np.ndindex(net_params['beta' + str(l)].shape):
                net_try = copy.deepcopy(net_params)
                net_try['beta' + str(l)][i] -= h
                c1 = compute_cost(X, Y, net_try, lambda_)

                net_try = copy.deepcopy(net_params)
                net_try['beta' + str(l)][i] += h
                c2 = compute_cost(X, Y, net_try, lambda_)

                grads['beta' + str(l)][i] = (c2 - c1) / (2 * h)
        
    return grads


def numerical_gradient(X, Y, parameters, lambda_):
    h = 1e-6
    G_Ws = []
    G_bs = []
    G_gammas = []
    G_betas = []
    c = compute_cost(X, Y, parameters, lambda_)

    for idx, l in enumerate(parameters):
        g_w = np.zeros_like(l[0])
        g_b = np.zeros_like(l[1])
        w_save = np.copy(l[0])
        b_save = np.copy(l[1])

        for i in range(len(l[1])):
            l[1] = b_save.copy()
            l[1][i] += h
            c2 = compute_cost(X, Y, parameters, lambda_)
            g_b[i] = (c2-c) / h

        l[1] = b_save

        for i in np.ndindex(l[0].shape):
            l[0] = w_save.copy()
            l[0][i] += h
            c2 = compute_cost(X, Y, parameters, lambda_)
            g_w[i] = (c2-c) / h

        l[0] = w_save
        G_Ws.append(g_w)
        G_bs.append(g_b)

        if parameters['use_bn'] and idx < len(parameters)-1:
            g_gamma = np.zeros_like(l[2])
            g_beta = np.zeros_like(l[3])
            gamma_save = np.copy(l[2])
            beta_save = np.copy(l[3])

            for i in range(len(l[2])):
                l[2] = gamma_save.copy()
                l[2][i] += h
                c2 = compute_cost(X, Y, parameters, lambda_)
                g_gamma[i] = (c2 - c) / h

            l[2] = gamma_save

            for i in range(len(l[3])):
                l[3] = beta_save.copy()
                l[3][i] += h
                c2 = compute_cost(X, Y, parameters, lambda_)
                g_beta[i] = (c2 - c) / h

            l[3] = beta_save
            G_gammas.append(g_gamma)
            G_betas.append(g_beta)

    return G_Ws, G_bs, G_gammas, G_betas



In [49]:
def load(filename):
    data = LoadBatch(filename)
    
    X = data[b'data'].astype(np.float32).reshape(-1, 3072).T / 255
    y = np.array(data[b'labels']).astype(np.int32)
    
    Y = np.zeros((10, y.shape[0]), dtype=np.float32)
    for i, label in enumerate(y):
        Y[label, i] = 1
    
    return X, Y, y + 1

def normalize_data(train, validation, test):
    # Compute the mean and standard deviation of the training data
    train_mean = np.mean(train, axis=1, keepdims=True)
    train_std = np.std(train, axis=1, keepdims=True)

    # Normalize the training, validation and test data
    train_norm = (train - train_mean) / train_std
    validation_norm = (validation - train_mean) / train_std
    test_norm = (test - train_mean) / train_std

    return train_norm, validation_norm, test_norm


# 3.1
train_X, train_Y, train_y = load('data_batch_1')
val_X, val_Y, val_y = load('data_batch_2')
test_X, test_Y, test_y = load('test_batch')

train_X_norm, val_X_norm, test_X_norm = normalize_data(train_X, val_X, test_X)
# initialize parameters with output of parameters numpyy array with keys: 'W', 'b', 'gammas', 'betas', and 'use_bn'
def initialize_parameters(layer_dims, use_bn=False):
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.normal(0, 0.001, (layer_dims[l], layer_dims[l-1]))
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    if use_bn:
        parameters['gammas'] = [np.ones((layer_dims[l], 1)) for l in range(1, L)]
        parameters['betas'] = [np.zeros((layer_dims[l], 1)) for l in range(1, L)]
        parameters['use_bn'] = True
    else:
        parameters['use_bn'] = False
        
    return parameters


"""def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters"""





"def initialize_parameters(layer_dims):\n    parameters = {}\n    L = len(layer_dims)            # number of layers in the network\n\n    for l in range(1, L):\n        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])\n        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))\n        \n    return parameters"

In [73]:
def affine_forward(x, w, b):
    out = None
    out = np.dot(w, x) + b
    cache = (x, w, b)
    return out, cache

def relu_forward(x):
    out = None
    out = np.maximum(0, x)
    cache = x
    return out, cache

def affine_relu_forward(x, w, b):
    out, cache = None, None
    a, fc_cache = affine_forward(x, w, b)
    out, relu_cache = relu_forward(a)
    cache = (fc_cache, relu_cache)
    return out, cache

def compute_cost(X, Y, parameters, lambda_):
    m = X.shape[1]
    L = len(parameters) // 2
    
    # Forward propagation
    A = X
    for l in range(1, L):
        A_prev = A
        A, _ = affine_relu_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
    
    scores, _ = affine_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])
    
    # Compute cost
    data_loss = np.sum(-np.log(np.sum(Y * scores, axis=0))) / m
    reg_loss = 0
    for l in range(1, L + 1):
        reg_loss += np.sum(parameters['W' + str(l)] ** 2)
    reg_loss *= lambda_ / (2 * m)
    
    return data_loss + reg_loss

def compute_accuracy(X, y, parameters):
    m = X.shape[1]
    L = len(parameters) // 2
    
    # Forward propagation
    A = X
    for l in range(1, L):
        A_prev = A
        A, _ = affine_relu_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
    
    scores, _ = affine_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])
    
    predicted_class = np.argmax(scores, axis=0)
    return np.mean(predicted_class == (y - 1))


In [52]:
parameters = initialize_parameters([3072, 50, 10], use_bn=False)
grads = compute_grads_num_slow(train_X_norm[:, :100], train_Y[:, :100],parameters,lambda_=0, h=1e-6)
print(grads)

/var/folders/7k/dhbxrqm54q998rq4v807swzr0000gn/T/ipykernel_57182/4227729906.py:33: RuntimeWarning: invalid value encountered in log
  data_loss = np.sum(-np.log(np.sum(Y * scores, axis=0))) / m


{'W1': array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]]), 'b1': array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]]), 'W2': a

In [77]:

"""def compute_gradients(X, Y, parameters, lambda_):
    grads = {}
    m = X.shape[1]
    L = len(parameters) // 2
    
    # Forward propagation
    A = X
    A_cache = [None] * L
    for l in range(1, L):
        A_prev = A
        A, cache = affine_relu_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        A_cache[l] = cache
    
    scores, cache = affine_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])
    
    # Backpropagation
    dA_prev = -Y / np.sum(Y * scores, axis=0)
    dA, dW, db = affine_backward(dA_prev, cache)
    grads['W' + str(L)] = dW + lambda_ * parameters['W' + str(L)] / m
    grads['b' + str(L)] = db
    
    for l in reversed(range(1, L)):
        dA_prev, dW, db = affine_relu_backward(dA, A_cache[l])
        grads['W' + str(l)] = dW + lambda_ * parameters['W' + str(l)] / m
        grads['b' + str(l)] = db
    
    return grads"""

def compute_gradients(X, Y, parameters, lambda_):
    grads = {}
    m = X.shape[1]
    L = len(parameters) // 2

    # Forward propagation
    A = X
    A_cache = [None] * L
    for l in range(1, L):
        A_prev = A
        A, cache = affine_relu_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        A_cache[l-1] = cache

    scores, cache = affine_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])

    # Backpropagation
    dA_prev = -Y / np.sum(Y * scores, axis=0)
    dA, dW, db = affine_backward(dA_prev, cache)
    grads['W' + str(L)] = dW + lambda_ * parameters['W' + str(L)] / m
    grads['b' + str(L)] = db

    for l in reversed(range(1, L)):
        dA_prev, dW, db = affine_relu_backward(dA, A_cache[l-1])
        grads['W' + str(l)] = dW + lambda_ * parameters['W' + str(l)] / m
        grads['b' + str(l)] = db



    return grads


def affine_backward(dout, cache):
    x, w, b = cache
    dx, dw, db = None, None, None
    dx = np.dot(w.T, dout)
    dw = np.dot(dout, x.T)
    db = np.sum(dout, axis=1, keepdims=True)
    return dx, dw, db

def relu_backward(dout, cache):
    dx, x = None, cache
    dx = dout * (x > 0)
    return dx

def affine_relu_backward(dout, cache):
    fc_cache, relu_cache = cache
    da = relu_backward(dout, relu_cache)
    dx, dw, db = affine_backward(da, fc_cache)
    return dx, dw, db

In [91]:
# compare gradients
def compare_gradients(X, Y, parameters, lambda_):
    L = len(parameters) // 2
    # Compute gradients using backpropagation
    grads = compute_gradients(X, Y, parameters, lambda_)
    
    # Compute gradients using numerical gradient approximation
    num_grads = compute_grads_num_slow(X, Y, parameters, lambda_, h=1e-6)
    
    # Loop over each layer and compare gradients
    for l in range(1, L+1):
        print(f'Layer {l}:')
        grad_w = np.array(grads['W' + str(l)])
        num_grad_w = np.array(num_grads['W' + str(l)])
        diff_w = num_grad_w - grad_w
        print(f'\tW: mean={np.mean(grad_w)} std={np.std(grad_w)} mean_diff={np.mean(diff_w)} std_diff={np.std(diff_w)} max_diff={np.max(diff_w)}')

        grad_b = np.array(grads['b' + str(l)])
        num_grad_b = np.array(num_grads['b' + str(l)])
        diff_b = num_grad_b - grad_b
        print(f'\tb: mean={np.mean(grad_b)} std={np.std(grad_b)} mean_diff={np.mean(diff_b)} std_diff={np.std(diff_b)} max_diff={np.max(diff_b)}')
    
    return grads, num_grads



# compare gradients, layers: 2
parameters2 = initialize_parameters([3072, 50, 10], use_bn=False)
grads2, num_grads2 = compare_gradients(train_X_norm[:, :100], train_Y[:, :100], parameters2, lambda_=0)


/var/folders/7k/dhbxrqm54q998rq4v807swzr0000gn/T/ipykernel_57182/4227729906.py:33: RuntimeWarning: invalid value encountered in log
  data_loss = np.sum(-np.log(np.sum(Y * scores, axis=0))) / m


Layer 1:
	W: mean=-19.660213502596363 std=481.46914056946076 mean_diff=nan std_diff=nan max_diff=nan
	b: mean=-119.33349425135877 std=510.2957225637157 mean_diff=nan std_diff=nan max_diff=nan
Layer 2:
	W: mean=-3930.3092677894383 std=8187.963736351885 mean_diff=nan std_diff=nan max_diff=nan
	b: mean=-189781.84195733376 std=207836.86097300384 mean_diff=nan std_diff=nan max_diff=nan


In [82]:
print(grads2)

{'W2': array([[ 5.48218804e+02,  2.20135106e+02,  3.63533054e+03,
         5.09108937e+01,  2.15873855e+03,  4.65056262e+01,
         1.01052309e+03,  1.36199080e+03,  4.92376163e+02,
         6.53963162e+02, -2.82600413e+02, -7.69621593e+01,
         1.61074043e+02, -1.52912856e+01, -7.33006352e+01,
         3.35400078e+02, -3.29425417e+02, -4.08817668e+02,
         1.25052467e+03, -1.64882603e+02, -3.55493967e+02,
         1.68592819e+03,  1.43686931e+02,  8.96924357e+02,
        -2.69653700e+02,  1.45107918e+02, -6.18293146e+01,
        -2.73035453e+02, -2.60231276e+02,  2.94054749e+02,
         1.24860620e+02, -3.48093291e+02,  1.77242238e+03,
        -7.07680058e+02,  4.84974124e+02, -2.28554037e+01,
         1.86368327e+02, -2.36158223e+02, -1.25637355e+03,
        -4.90382276e+02, -2.62568261e+02, -8.42922397e+02,
         1.17645355e+03,  5.47520788e+02,  4.27594148e+02,
         4.75610688e+02,  1.93543303e+02,  3.51817643e+02,
         5.73964769e+02, -2.20129525e+02],
      

In [ ]:

# compare gradients, layers: 3
parameters3 = initialize_parameters([3072, 50, 50, 10], use_bn=False)
grads3, num_grads3 = compare_gradients(train_X_norm[:, :100], train_Y[:, :100], parameters3, lambda_=0)

# compare gradients, layers: 4
parameters4 = initialize_parameters([3072, 50, 50, 50, 10], use_bn=False)
grads4, num_grads4 = compare_gradients(train_X_norm[:, :100], train_Y[:, :100], parameters4, lambda_=0)

